In [6]:
filepath = "data/Supply chain logistics problem.xlsx"

In [7]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# Load dataset with error handling
try:
    order_list = pd.read_excel(
        filepath, sheet_name="OrderList"
    )
    freight_rates = pd.read_excel(
        filepath, sheet_name="FreightRates"
    )
    wh_costs = pd.read_excel(
        filepath, sheet_name="WhCosts"
    )
    wh_capacities = pd.read_excel(
        filepath, sheet_name="WhCapacities"
    )
    products_per_plant = pd.read_excel(
        filepath, sheet_name="ProductsPerPlant"
    )
    plant_ports = pd.read_excel(
        filepath, sheet_name="PlantPorts"
    )
    logging.info("Datasets loaded successfully.")
except FileNotFoundError as e:
    logging.error(f"Error loading dataset: {e}")
    raise

# Aggregate demand by destination port
demand_by_port = order_list.groupby("Destination Port")["Unit quantity"].sum().to_dict()

# Extract plant capacities
plant_capacities = wh_capacities.set_index("Plant ID")["Daily Capacity "].to_dict()

# Map plants to their respective ports
plant_to_port = plant_ports.set_index("Plant Code")["Port"].to_dict()

# Filter relevant freight rates based on available plant-port combinations
relevant_freight_rates = freight_rates[
    freight_rates["orig_port_cd"].isin(plant_to_port.values())
    & freight_rates["dest_port_cd"].isin(demand_by_port.keys())
]

# Create cost dictionary for relevant (origin, destination) pairs
freight_costs = (
    relevant_freight_rates.groupby(["orig_port_cd", "dest_port_cd"])["minimum cost"]
    .min()
    .to_dict()
)

# Ensure valid pairs exist
valid_pairs = [
    (plant, "PORT09")
    for plant, port in plant_to_port.items()
    if (port, "PORT09") in freight_costs
]
if not valid_pairs:
    logging.error("No valid (plant, port) pairs found in the dataset.")
    raise ValueError("Invalid dataset: No valid routes available.")

cost_vector = [
    freight_costs[(plant_to_port[plant], "PORT09")] for plant, _ in valid_pairs
]

# Construct inequality constraints for plant capacities
A_ub = [
    [1 if pair[0] == plant else 0 for pair in valid_pairs]
    for plant in plant_capacities.keys()
]
b_ub = [plant_capacities[plant] for plant in plant_capacities.keys()]

# Equality constraint to satisfy demand at PORT09
A_eq = [
    [1 for _ in valid_pairs]
]  # All pairs contribute to meeting the demand at PORT09
b_eq = [sum(plant_capacities.values())]  # Adjusted demand to match total supply

# Solve the linear programming problem with adjusted demand
result = linprog(
    cost_vector, A_ub=A_ub, b_ub=b_ub, A_eq=A_eq, b_eq=b_eq, method="highs"
)

# Output results if successful
if result.success:
    optimal_solution = {
        valid_pairs[i]: result.x[i] for i in range(len(valid_pairs)) if result.x[i] > 0
    }
    total_cost = result.fun
    logging.info("Optimization completed successfully.")
    print("Optimal solution:\n", optimal_solution)
    print("Total cost:\n", total_cost)
else:
    logging.error("No optimal solution found.")
    print("No optimal solution found.")

# Validate result
if abs(sum(result.x) - sum(plant_capacities.values())) > 1e-6:
    logging.warning("Warning: Solution may not fully meet the demand.")

INFO:root:Datasets loaded successfully.
INFO:root:Optimization completed successfully.


Optimal solution:
 {('PLANT01', 'PORT09'): 1070.0, ('PLANT02', 'PORT09'): 138.0, ('PLANT03', 'PORT09'): 1013.0, ('PLANT04', 'PORT09'): 554.0, ('PLANT05', 'PORT09'): 385.0, ('PLANT06', 'PORT09'): 49.0, ('PLANT07', 'PORT09'): 265.0, ('PLANT08', 'PORT09'): 14.0, ('PLANT09', 'PORT09'): 11.0, ('PLANT10', 'PORT09'): 118.0, ('PLANT11', 'PORT09'): 332.0, ('PLANT12', 'PORT09'): 209.0, ('PLANT13', 'PORT09'): 490.0, ('PLANT14', 'PORT09'): 549.0, ('PLANT15', 'PORT09'): 11.0, ('PLANT16', 'PORT09'): 457.0, ('PLANT17', 'PORT09'): 8.0, ('PLANT18', 'PORT09'): 111.0, ('PLANT19', 'PORT09'): 7.0}
Total cost:
 23146.711600000002
